<a href="https://colab.research.google.com/github/shravanc/tf/blob/master/adult_data_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [0]:
URL = 'https://raw.githubusercontent.com/shravanc/tf/master/tensorflow_api_tutorials/adult/adult.data.csv'
df  = pd.read_csv(URL)
df.head()

,age,workclass,fnlwgt,education,education_num,martial_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,good_salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


In [0]:
train, test = train_test_split(df, test_size=0.2)
train, val  = train_test_split(train, test_size=0.2)


In [0]:
def df_to_dataset(df, shuffle=True, batch_size=32):
  df = df.copy()
  labels = df.pop('good_salary')
  ds = tf.data.Dataset.from_tensor_slices((dict(df) , labels))
  if shuffle:
    ds.shuffle(buffer_size=len(df))
  ds = ds.batch(batch_size)
  return ds


In [0]:
batch_size=32
train_ds = df_to_dataset(train, shuffle=True , batch_size=batch_size)
val_ds   = df_to_dataset(val  , shuffle=False, batch_size=batch_size)
test_ds  = df_to_dataset(test , shuffle=False, batch_size=batch_size)


In [0]:
for train_batch, label_batch in train_ds.take(1):
  print(list(train_batch.keys()))

['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'martial_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country']


In [0]:
feature_columns = []

In [0]:
feature_columns = []
numeric_headers = ['age', 'education_num', 'fnlwgt', 'capital_gain', 'capital_loss', 'hours_per_week']
for header in numeric_headers:
  feature_columns.append(feature_column.numeric_column(header))


In [0]:
# embedding
# native_country
country = feature_column.categorical_column_with_vocabulary_list('native_country', df['native_country'].unique())
country_embedding = feature_column.embedding_column(country, dimension=40)
feature_columns.append(country_embedding)
#demo(country_embedding)

occupation = feature_column.categorical_column_with_vocabulary_list('occupation', df['occupation'].unique())
occupation_embedding = feature_column.embedding_column(occupation, dimension=10)
feature_columns.append(occupation_embedding)
#demo(occupation_embedding)

education = feature_column.categorical_column_with_vocabulary_list('education', df['education'].unique())
education_embedding = feature_column.embedding_column(education, dimension=20)
feature_columns.append(education_embedding)


In [0]:
#indicator cols
relationship = feature_column.categorical_column_with_vocabulary_list('relationship', df['relationship'].unique())
relationship_one_hot = feature_column.indicator_column(relationship)
feature_columns.append(relationship_one_hot)

#sex
sex = feature_column.categorical_column_with_vocabulary_list('sex', df['sex'].unique())
sex_one_hot = feature_column.indicator_column(sex)
feature_columns.append(sex_one_hot)

#race
race = feature_column.categorical_column_with_vocabulary_list('race', df['race'].unique())
race_one_hot = feature_column.indicator_column(race)
feature_columns.append(race_one_hot)

#workclass
workclass = feature_column.categorical_column_with_vocabulary_list('workclass', df['workclass'].unique())
workclass_one_hot = feature_column.indicator_column(workclass)
feature_columns.append(workclass_one_hot)

#bucketize cols
age = feature_column.numeric_column('age')
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
feature_columns.append(age_buckets)

hours_per_week = feature_column.numeric_column('hours_per_week')
hours_per_week_bucketize = feature_column.bucketized_column(hours_per_week, boundaries=[15, 20, 25, 30, 35, 40, 45, 50, 60])
feature_columns.append(hours_per_week_bucketize)

#hashed bucket
martial_status_hashed = feature_column.categorical_column_with_hash_bucket(
                  'martial_status', hash_bucket_size=1000)
martial_status_one_hot = feature_column.indicator_column(martial_status_hashed)
feature_columns.append(martial_status_one_hot)

**Building Model**

In [0]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

model = tf.keras.Sequential([
        feature_layer,
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_ds, validation_data=val_ds, epochs=10)

loss, accuracy = model.evaluate(test_ds)
print("Accuracy---->", accuracy)


Train for 652 steps, validate for 163 steps
Epoch 1/10
652/652 [==============================] - 8s 13ms/step - loss: 181.8839 - accuracy: 0.6723 - val_loss: 159.7808 - val_accuracy: 0.7898
Epoch 2/10
652/652 [==============================] - 7s 11ms/step - loss: 68.4160 - accuracy: 0.6946 - val_loss: 45.9904 - val_accuracy: 0.7985
Epoch 3/10
652/652 [==============================] - 7s 10ms/step - loss: 38.6588 - accuracy: 0.6996 - val_loss: 14.1748 - val_accuracy: 0.7958
Epoch 4/10
652/652 [==============================] - 7s 10ms/step - loss: 24.3752 - accuracy: 0.7172 - val_loss: 5.2974 - val_accuracy: 0.7649
Epoch 5/10
652/652 [==============================] - 7s 10ms/step - loss: 13.7516 - accuracy: 0.7295 - val_loss: 4.6773 - val_accuracy: 0.7668
Epoch 6/10
652/652 [==============================] - 7s 10ms/step - loss: 11.7559 - accuracy: 0.7294 - val_loss: 6.4269 - val_accuracy: 0.8006
Epoch 7/10
652/652 [==============================] - 7s 11ms/step - loss: 9.0754 - acc